In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| default_exp v2.nixtla_client

# V2 Nixtla Client

In [ ]:
#| export
import logging
import math
import os
import warnings
from itertools import chain
from typing import Any, List, Literal, Mapping, Optional, Union

import httpx
import numpy as np
import pandas as pd
import utilsforecast.processing as ufp
from pydantic import NonNegativeInt, PositiveInt
from utilsforecast.compat import DataFrame, pl_DataFrame, pl_Series
from utilsforecast.validation import validate_format, validate_freq

from nixtla.core.api_error import ApiError
from nixtla.core.http_client import HttpClient
from nixtla.utils import _restrict_input_samples

In [ ]:
#| export
logger = logging.getLogger(__name__)

In [ ]:
#| exporti
_LOSS = Literal["default", "mae", "mse", "rmse", "mape", "smape"]
_MODEL = Literal["timepgt-1", "timegpt-1-long-horizon"]

def _tail(proc: ufp.ProcessedDF, n: int) -> ufp.ProcessedDF:
    n_series = proc.indptr.size - 1
    new_sizes = np.minimum(np.diff(proc.indptr), n)
    new_indptr = np.append(0, new_sizes.cumsum())
    new_data = np.empty_like(proc.data, shape=(new_indptr[-1], proc.data.shape[1]))
    for i in range(n_series):
        new_data[new_indptr[i] : new_indptr[i + 1]] = proc.data[
            proc.indptr[i + 1] - new_sizes[i] : proc.indptr[i + 1]
        ]
    return ufp.ProcessedDF(
        uids=proc.uids,
        last_times=proc.last_times,
        data=new_data,
        indptr=new_indptr,
        sort_idxs=None,
    )

In [ ]:
#| export
class NixtlaClient:

    def __init__(
        self,
        api_key: Optional[str] = None,
        base_url: Optional[str] = None,
        timeout: int = 60,
        max_retries: int = 6,
    ):
        if api_key is None:
            api_key = os.environ['NIXTLA_API_KEY']
        if base_url is None:
            base_url = os.getenv('NIXTLA_BASE_URL', 'https://dashboard.nixtla.io/api')
        self._client_kwargs = {
            'base_url': base_url,
            'headers': {'Authorization': f'Bearer {api_key}'},
            'timeout': timeout,
        }
        self.max_retries = max_retries
        self._model_params: Mapping[str, Tuple[int]] = {}

    def _make_request(self, endpoint: str, payload: Mapping[str, any]):
        with httpx.Client(**self._client_kwargs) as httpx_client:
            client = HttpClient(httpx_client=httpx_client)
            resp = client.request(
                method='post',
                url=endpoint,
                json=payload,
                max_retries=self.max_retries,
            )
            resp_body = resp.json()
            if resp.status_code != 200:
                raise ApiError(status_code=resp.status_code, body=resp_body)
        if 'data' in resp_body:
            resp_body = resp_body['data']
        return resp_body

    def _get_model_params(self, freq: str, model: str):
        key = (freq, model)
        if key not in self._model_params:
            payload = {'freq': freq, 'model': model}
            params = self._make_request('/model_params', payload)['detail']
            self._model_params[key] = (params['input_size'], params['horizon'])
        return self._model_params[key]

    def forecast(
        self,
        df: DataFrame,
        h: PositiveInt,
        freq: str,    
        id_col: str = 'unique_id',
        time_col: str = 'ds',
        target_col: str = 'y',
        X_df: Optional[DataFrame] = None,
        level: Optional[List[Union[int, float]]] = None,
        finetune_steps: NonNegativeInt = 0,
        finetune_loss: _LOSS = 'default',
        clean_ex_first: bool = True,
        model: _MODEL = 'timegpt-1',
    ):
        """Forecast your time series using TimeGPT.

        Parameters
        ----------
        df : pandas or polars DataFrame
            dataframe to forecast. Must have at least [id_col, time_col, target_col] columns.
        h : int
            Forecast horizon.
        freq : str
            pandas or polars frequency string.
        id_col : str (default='unique_id')
            Column that identifies each serie.
        time_col : str (default='ds')
            Column that identifies each timestep, its values must be timestamps.
        target_col : str (default='y')
            Column that contains the target.
        X_df : pandas.DataFrame, optional (default=None)
            DataFrame with [id_col, time_col] columns and `df`'s future exogenous.
        level : List[float], optional (default=None)
            Confidence levels between 0 and 100 for prediction intervals.
        finetune_steps : int (default=0)
            Number of steps used to finetune learning TimeGPT in the
            new data.
        finetune_loss : str (default='default')
            Loss function to use for finetuning. Options are: `default`, `mae`, `mse`, `rmse`, `mape`, and `smape`.
        clean_ex_first : bool (default=True)
            Clean exogenous signal before making forecasts
            using TimeGPT.
        model : str (default='timegpt-1')
            Model to use as a string. Options are: `timegpt-1`, and `timegpt-1-long-horizon`. 
            We recommend using `timegpt-1-long-horizon` for forecasting 
            if you want to predict more than one seasonal 
            period given the frequency of your data.
        
        Returns
        -------
        fcsts_df : pandas.DataFrame
            DataFrame with TimeGPT forecasts for point predictions and probabilistic
            predictions (if level is not None).
        """
        logger.info('Validating inputs...')
        validate_format(df=df, id_col=id_col, time_col=time_col, target_col=target_col)
        validate_freq(times=df[time_col], freq=freq)
        exog_df = [c for c in df.columns if c not in (id_col, time_col, target_col)]
        if X_df is not None:
            exog_X_df = [c for c in X_df.columns if c not in (id_col, time_col)]
            missing_df = set(exog_X_df) - set(exog_df)
            if missing_df:
                raise ValueError(
                    'The following exogenous features are present in `X_df` '
                    f'but not in `df`: {missing_df}.'
                )
            missing_X_df = set(exog_df) - set(exog_X_df)
            if missing_X_df:
                raise ValueError(
                    'The following exogenous features are present in `df` '
                    f'but not in `X_df`: {missing_X_df}.'
                )
            if exog_df != exog_X_df:
                # rearrange columns if necessary
                X_df = X_df[[id_col, time_col, *x_cols]]
            x_cols = exog_df
        else:
            if exog_df:
                warnings.warn(
                    f'`df` contains the following exogenous features: {exog_df}, '
                    'but `X_df` was not provided. They will be ignored.'
                )
                df = df[[id_col, time_col, target_col]]
            x_cols = None

        logger.info('Preprocessing dataframes...')
        processed = ufp.process_df(
            df=df, id_col=id_col, time_col=time_col, target_col=target_col
        )
        if X_df is not None:
            processed_X = ufp.process_df(
                df=X_df, id_col=id_col, time_col=time_col, target_col=None,
            )
            X_future = processed_X.data.T.tolist()
        else:
            X_future = None

        model_input_size, model_horizon = self._get_model_params(freq, model)
        restrict_input = finetune_steps == 0 and X_df is None
        if restrict_input:
            logger.info('Restricting input...')
            new_input_size = _restrict_input_samples(
                level=level,
                input_size=model_input_size,
                model_horizon=model_horizon,
                h=h,
            )
            processed = _tail(processed, new_input_size)
        if processed.data.shape[1] > 1:
            X = processed.data[:, 1:].T.tolist()
        else:
            X = None
        payload = {
            'series': {
                'y': processed.data[:, 0].tolist(),
                'sizes': np.diff(processed.indptr).tolist(),
                'X': X,
                'X_future': X_future,
            },
            'model': model,
            'h': h,
            'freq': freq,
            'clean_ex_first': clean_ex_first,
            'level': level,
            'finetune_steps': finetune_steps,
            'finetune_loss': finetune_loss,
        }

        logger.info('Calling Forecast Endpoint...')
        resp = self._make_request('/v2/forecast', payload)

        # assemble result
        out = ufp.make_future_dataframe(
            uids=processed.uids,
            last_times=type(processed.uids)(processed.last_times),
            freq=freq,
            h=h,
            id_col=id_col,
            time_col=time_col,
        )
        out = ufp.assign_columns(out, 'TimeGPT', resp['mean'])
        if resp['intervals'] is not None:
            intervals_df = type(df)(
                {
                    f'TimeGPT-{k}': v for k, v in resp['intervals'].items()
                }
            )
            out = ufp.horizontal_concat([out, intervals_df])
        if resp['weights_x'] is not None:
            self.weights_x = type(df)({
                'features': x_cols,
                'weights': resp['weights_x'],
            })
        return out

In [ ]:
import os
os.environ['NIXTLA_BASE_URL'] = 'http://localhost:8000'

In [ ]:
from utilsforecast.data import generate_series

import polars as pl

In [ ]:
series = generate_series(4, min_length=40, max_length=200, n_static_features=2, static_as_categorical=False)
series['unique_id'] = series['unique_id'].astype('int64')
valid = series.groupby('unique_id', observed=True).tail(20)
train = series.drop(valid.index)
train_pl = pl.from_pandas(train)
valid_pl = pl.from_pandas(valid)

In [ ]:
client = NixtlaClient(max_retries=0)

In [ ]:
client.forecast(df=train, h=20, freq="D", level=[80])

In [ ]:
client.forecast(df=train, X_df=valid.drop(columns='y'), h=20, freq="D", level=[80, 90.5])

In [ ]:
client.forecast(df=series, h=2, freq="D", level=[80])

In [ ]:
client.forecast(df=train_pl, X_df=valid_pl.drop('y'), h=20, freq="1d", level=[80])

In [ ]:
from nixtla import NixtlaClient as NixtlaV1

In [ ]:
v1_client = NixtlaV1()

In [ ]:
v1_client.forecast(df=series, h=2, freq="D", level=[80])